In [2]:
import os
import tarfile
import urllib

import pandas as pd
from sklearn.metrics import mean_squared_error as mse

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.datasets import load_digits
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn import metrics
from sklearn.model_selection import train_test_split

#reading ARP MitM dataset
PATH = "arp_mitm"

FILE = "ARP MitM_dataset-002.csv"
L_FILE = "ARP MitM_labels.csv"

#input data -> measured
csv_path = os.path.join(PATH, FILE)
dataset = pd.read_csv(csv_path, header=None)  

dataset = dataset.dropna()
display(dataset.head())
display(dataset.info())
display(dataset.describe())

,0,1,2,3,4,5,6,7,8,9,...,105,106,107,108,109,110,111,112,113,114
0,1.000000,1294.0,0.000000e+00,1.000000,1294.0,0.000000e+00,1.000000,1294.0,0.000000e+00,1.000000,...,0.000000e+00,0.0,0.0,1.000000,1294.0,0.000000,1294.0,0.000000e+00,0.0,0.0
1,1.000000,1514.0,0.000000e+00,1.000000,1514.0,0.000000e+00,1.000000,1514.0,0.000000e+00,1.000000,...,0.000000e+00,0.0,0.0,1.000000,1514.0,0.000000,1514.0,0.000000e+00,0.0,0.0
2,1.999505,1294.0,6.984919e-10,1.999703,1294.0,2.328306e-10,1.999901,1294.0,6.984919e-10,1.999990,...,0.000000e+00,0.0,0.0,1.999999,1294.0,0.000000,1294.0,0.000000e+00,0.0,0.0
3,2.998985,1294.0,9.313226e-10,2.999391,1294.0,4.656613e-10,2.999797,1294.0,6.984919e-10,2.999980,...,6.984919e-10,0.0,0.0,2.999998,1294.0,0.000015,1294.0,2.328306e-10,0.0,0.0
4,3.998061,1294.0,9.313226e-10,3.998836,1294.0,2.328306e-10,3.999612,1294.0,6.984919e-10,3.999961,...,2.328306e-10,0.0,0.0,3.999996,1294.0,0.000000,1294.0,0.000000e+00,0.0,0.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2504267 entries, 0 to 2504266
Columns: 115 entries, 0 to 114
dtypes: float64(115)
memory usage: 2.2 GB


None

,0,1,2,3,4,5,6,7,8,9,...,105,106,107,108,109,110,111,112,113,114
count,2.504267e+06,2.504267e+06,2.504267e+06,2.504267e+06,2.504267e+06,2.504267e+06,2.504267e+06,2.504267e+06,2.504267e+06,2.504267e+06,...,2.504267e+06,2.504267e+06,2.504267e+06,2.504267e+06,2.504267e+06,2.504267e+06,2.504267e+06,2.504267e+06,2.504267e+06,2.504267e+06
mean,3.161843e+02,1.263429e+03,1.629564e+05,5.206959e+02,1.263908e+03,1.624291e+05,1.542137e+03,1.264399e+03,1.618282e+05,1.508207e+04,...,1.618792e+05,-2.279080e-03,-7.277519e-07,2.433348e+04,1.264685e+03,3.848150e+02,1.319794e+03,1.617353e+05,-2.131455e-03,-5.309278e-07
std,1.261311e+02,2.731945e+02,4.387131e+04,2.018029e+02,2.730516e+02,4.250538e+04,5.917664e+02,2.729755e+02,4.140203e+04,5.965275e+03,...,4.570896e+04,1.849400e+00,6.071145e-04,6.679195e+03,2.771983e+02,1.050558e+02,7.498082e+01,4.563698e+04,1.538249e+00,5.666412e-04
min,1.000000e+00,6.000000e+01,0.000000e+00,1.000000e+00,6.000000e+01,0.000000e+00,1.000000e+00,6.000000e+01,0.000000e+00,1.000000e+00,...,0.000000e+00,-1.464452e+03,-4.688243e-01,1.000000e+00,6.000000e+01,0.000000e+00,6.000000e+01,0.000000e+00,-8.329919e+02,-2.887023e-01
25%,1.893583e+02,1.302018e+03,1.490105e+05,3.071747e+02,1.301232e+03,1.466209e+05,8.890711e+02,1.297967e+03,1.435192e+05,8.741032e+03,...,1.793071e+05,0.000000e+00,0.000000e+00,2.347740e+04,1.335240e+03,4.248047e+02,1.335240e+03,1.804590e+05,0.000000e+00,0.000000e+00
50%,3.604483e+02,1.328082e+03,1.727128e+05,6.178137e+02,1.331693e+03,1.754399e+05,1.921768e+03,1.336063e+03,1.789780e+05,1.945002e+04,...,1.809940e+05,0.000000e+00,0.000000e+00,2.763519e+04,1.340819e+03,4.254162e+02,1.340819e+03,1.809789e+05,0.000000e+00,0.000000e+00
75%,4.041041e+02,1.342751e+03,1.882200e+05,6.662270e+02,1.342305e+03,1.863478e+05,1.974742e+03,1.341165e+03,1.843493e+05,1.964070e+04,...,1.821485e+05,0.000000e+00,0.000000e+00,2.802381e+04,1.341263e+03,4.259399e+02,1.341263e+03,1.814248e+05,0.000000e+00,0.000000e+00
max,5.365877e+02,1.514000e+03,4.942291e+05,8.073135e+02,1.514000e+03,4.948259e+05,2.124893e+03,1.514000e+03,4.953075e+05,1.983936e+04,...,4.257562e+05,7.038125e+01,8.482200e-02,3.091260e+04,1.514000e+03,6.525000e+02,1.514000e+03,4.257562e+05,3.335587e+02,1.560442e-01


In [3]:
#Reading output data is the observation of malicous and none malicous 
csv_path_L = os.path.join(PATH, L_FILE)

dataset_L = pd.read_csv(csv_path_L, dtype={"": int, "x": 'float64'})  

display(dataset_L.head())
display(dataset_L.info())
display(dataset_L.describe())


,Unnamed: 0,x
0,1,0.0
1,2,0.0
2,3,0.0
3,4,0.0
4,5,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2504267 entries, 0 to 2504266
Data columns (total 2 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   x           float64
dtypes: float64(1), int64(1)
memory usage: 38.2 MB


None

,Unnamed: 0,x
count,2.504267e+06,2.504267e+06
mean,1.252134e+06,4.573282e-01
std,7.229198e+05,4.981759e-01
min,1.000000e+00,0.000000e+00
25%,6.260675e+05,0.000000e+00
50%,1.252134e+06,0.000000e+00
75%,1.878200e+06,1.000000e+00
max,2.504267e+06,1.000000e+00


In [4]:
#counting how many malicous and non malicous was observed in the dataset
dataset_L["x"].value_counts()

0.0    1358995
1.0    1145272
Name: x, dtype: int64

In [5]:
#trying to look at the coleration between feature but clearly this does not help much since we have wait too much data.
#all the feature have some kind of coleration to one another as observe. This indicate that we cant fix one input while 
#changing the other inputs. 
correlation = dataset.corr()
correlation

,0,1,2,3,4,5,6,7,8,9,...,105,106,107,108,109,110,111,112,113,114
0,1.000000,0.582905,0.705597,0.990574,0.578305,0.749803,0.951491,0.572748,0.794169,0.891043,...,0.615960,0.002873,0.003038,0.166781,0.561470,0.650631,0.410455,0.617802,0.001119,0.000427
1,0.582905,1.000000,0.846365,0.596223,0.999789,0.876437,0.596959,0.999020,0.903938,0.575626,...,0.562599,0.005676,0.006000,0.416503,0.982971,0.842446,0.414742,0.563238,0.002580,0.001754
2,0.705597,0.846365,1.000000,0.749736,0.852366,0.993503,0.782079,0.859897,0.967417,0.767899,...,0.653403,0.004788,0.005091,0.324240,0.853774,0.824940,0.450215,0.651078,0.001933,0.001248
3,0.990574,0.596223,0.749736,1.000000,0.593351,0.786827,0.982412,0.589583,0.823499,0.927627,...,0.634783,0.002969,0.003136,0.171291,0.578531,0.670321,0.422658,0.636406,0.001254,0.000556
4,0.578305,0.999789,0.852366,0.593351,1.000000,0.880416,0.596083,0.999684,0.905530,0.575639,...,0.563512,0.005681,0.006005,0.416804,0.983866,0.843404,0.415233,0.564082,0.002593,0.001773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,0.650631,0.842446,0.824940,0.670321,0.843404,0.844340,0.675627,0.844135,0.859351,0.652328,...,0.913966,0.003632,0.003499,0.302025,0.896333,1.000000,0.626062,0.914487,0.004296,0.002850
111,0.410455,0.414742,0.450215,0.422658,0.415233,0.460515,0.425721,0.415466,0.468042,0.410591,...,0.672693,0.018758,0.018248,0.227333,0.511353,0.626062,1.000000,0.672269,0.021116,0.014354
112,0.617802,0.563238,0.651078,0.636406,0.564082,0.663210,0.640408,0.564764,0.671911,0.616766,...,0.999541,0.004216,0.004099,0.151225,0.643867,0.914487,0.672269,1.000000,0.004740,0.003200
113,0.001119,0.002580,0.001933,0.001254,0.002593,0.002007,0.001320,0.002596,0.002170,0.001240,...,0.004796,0.571747,0.538860,0.005047,0.005738,0.004296,0.021116,0.004740,1.000000,0.985749


In [6]:
# QDA modeling to see if non-linear model would fix better

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

#splitting ratio is 85:15 for training and testing
x_train, x_test, y_train, y_test = train_test_split( dataset, dataset_L, test_size=0.15, random_state=4)


qda = QuadraticDiscriminantAnalysis()
qda_training = qda.fit(x_train,(y_train['x']==1)).predict(x_train)
qda_testing = qda.fit(x_test,(y_test['x']==1)).predict(x_test)


print("\nTRAINING DATA:")
print(classification_report((y_train['x']==1), qda_training, target_names=['not malicous', 'malicous']))
print(metrics.accuracy_score((y_train['x']==1), qda_training))

print("\nTEST DATA:")
print(classification_report((y_test['x']==1), qda_testing, target_names=['not malicous', 'malicous']))
print(metrics.accuracy_score((y_test['x']==1), qda_testing))


TRAINING DATA:
              precision    recall  f1-score   support

not malicous       1.00      0.95      0.98   1155228
    malicous       0.95      1.00      0.97    973398

    accuracy                           0.97   2128626
   macro avg       0.97      0.98      0.97   2128626
weighted avg       0.98      0.97      0.97   2128626

0.9738131545889226

TEST DATA:
              precision    recall  f1-score   support

not malicous       0.99      0.94      0.97    203767
    malicous       0.94      0.99      0.96    171874

    accuracy                           0.96    375641
   macro avg       0.96      0.97      0.96    375641
weighted avg       0.97      0.96      0.96    375641

0.9643276426162214


In [7]:
print("FOR MODEL WITH NO FEATURE SELECTION")
print("Training MSE = ", mse(qda_training,y_train['x'].values))
print("Testing MSE = ", mse(qda_testing,y_test['x'].values))

FOR MODEL WITH NO FEATURE SELECTION
Training MSE =  0.02618684541107738
Testing MSE =  0.03567235738377866


In [8]:
#comment: from both QDA and LDA, the random splitting ratio 1:1 giving the best testing error while the training error
#barely change. This telling us that we might have been overfitting the model when having a bigger training data

In [9]:
#feature selection done by ExtraTreesClassifier()
feature_select = [12, 13, 27, 28, 58, 63, 77, 79, 108, 109]

x=dataset.iloc[:,feature_select]
x1=dataset
y=dataset_L.drop('Unnamed: 0', axis=1)

display(x.head(5))
display(y.head(5))

,12,13,27,28,58,63,77,79,108,109
0,1.000000,1294.0,1.000000,1294.0,1.000000,0.0,1.000000,0.000000e+00,1.000000,1294.0
1,1.000000,1514.0,1.000000,1514.0,1.000000,0.0,1.000000,0.000000e+00,1.000000,1514.0
2,1.999999,1294.0,1.999999,1294.0,1.999999,0.0,1.999999,5.642016e+17,1.999999,1294.0
3,2.999998,1294.0,2.999998,1294.0,2.999998,0.0,2.999998,5.015123e+17,2.999998,1294.0
4,3.999996,1294.0,3.999996,1294.0,3.999996,0.0,3.999996,4.231509e+17,3.999996,1294.0


,x
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [10]:
# QDA modeling with selected feature from ExtraTreesClassifier()
#splitting ratio is 3:1 for training and testing
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.33, random_state=4)


qda = QuadraticDiscriminantAnalysis()
qda_training = qda.fit(x_train,(y_train['x']==1)).predict(x_train)
qda_testing = qda.fit(x_test,(y_test['x']==1)).predict(x_test)


print("\nTRAINING DATA:")
print(classification_report((y_train['x']==1), qda_training, target_names=['not malicous', 'malicous']))
print(metrics.accuracy_score((y_train['x']==1), qda_training))

print("\nTEST DATA:")
print(classification_report((y_test['x']==1), qda_testing, target_names=['not malicous', 'malicous']))
print(metrics.accuracy_score((y_test['x']==1), qda_testing))

C:\Users\nickn\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")



TRAINING DATA:
              precision    recall  f1-score   support

not malicous       0.97      0.75      0.85    910697
    malicous       0.77      0.97      0.86    767161

    accuracy                           0.85   1677858
   macro avg       0.87      0.86      0.85   1677858
weighted avg       0.88      0.85      0.85   1677858

0.8510303017299438

TEST DATA:
              precision    recall  f1-score   support

not malicous       0.97      0.50      0.66    448298
    malicous       0.62      0.98      0.76    378111

    accuracy                           0.72    826409
   macro avg       0.80      0.74      0.71    826409
weighted avg       0.81      0.72      0.71    826409

0.7203091931477029


In [11]:
print("FOR MODEL WITH EXTRA TREES CLASSIFIER FEATURE SELECTION")
print("Training MSE = ", mse(qda_training,y_train['x'].values))
print("Testing MSE = ", mse(qda_testing,y_test['x'].values))

FOR MODEL WITH EXTRA TREES CLASSIFIER FEATURE SELECTION
Training MSE =  0.14896969827005624
Testing MSE =  0.2796908068522971


In [12]:
feature_select = [12, 13, 27, 28, 63, 56, 77, 88, 108, 101]

x=dataset.iloc[:,feature_select]
x1=dataset
y=dataset_L.drop('Unnamed: 0', axis=1)

x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.33, random_state=4)

qda = QuadraticDiscriminantAnalysis()
qda_training = qda.fit(x_train,(y_train['x']==1)).predict(x_train)
qda_testing = qda.fit(x_test,(y_test['x']==1)).predict(x_test)


print("\nTRAINING DATA:")
print(classification_report((y_train['x']==1), qda_training, target_names=['not malicous', 'malicous']))
print(metrics.accuracy_score((y_train['x']==1), qda_training))

print("\nTEST DATA:")
print(classification_report((y_test['x']==1), qda_testing, target_names=['not malicous', 'malicous']))
print(metrics.accuracy_score((y_test['x']==1), qda_testing))


TRAINING DATA:
              precision    recall  f1-score   support

not malicous       0.99      0.99      0.99    910697
    malicous       0.99      0.99      0.99    767161

    accuracy                           0.99   1677858
   macro avg       0.99      0.99      0.99   1677858
weighted avg       0.99      0.99      0.99   1677858

0.989407923674113

TEST DATA:
              precision    recall  f1-score   support

not malicous       0.84      1.00      0.91    448298
    malicous       1.00      0.78      0.87    378111

    accuracy                           0.90    826409
   macro avg       0.92      0.89      0.89    826409
weighted avg       0.91      0.90      0.90    826409

0.8982235188629359


In [13]:
print("FOR MODEL WITH SELECT K BEST FEATURE SELECTION")
print("Training MSE = ", mse(qda_training,y_train['x'].values))
print("Testing MSE = ", mse(qda_testing,y_test['x'].values))

FOR MODEL WITH SELECT K BEST FEATURE SELECTION
Training MSE =  0.010592076325886934
Testing MSE =  0.1017764811370641


In [14]:
#the selection model from the extratreeclasscification is that working well with the LDA and QDA, the main reason
#for this is that we do not have enough data for training set. 

In [15]:
#conclusion form the LDA and QDA
#1. the best model id QDA with fine tune splitting dataset between training and testing set as ratio 1:1
#2. QDA working better than LDA implied that the dataset have non-linear realtionship.
#3. QDA give best test error rate of  0.9788864450609919 and testing error 0.9786883661719642 which is very close to one.
# also the performence metrics indicate a very good result for both training and testing data sucj as presicion, recal, 
# and F1-score are all very hight (in range of .97 abd .98)